# Hello World of Pytorch

更新时间：2019/3/25(Part II未完成)

In [1]:
import torch

## Part I: Understand 'Autograd'

利用bp计算相应梯度时，原本仅有标量->张量的运算，若最终结果为张量，
则torch.autograd.backward()需要一个grad_tensors参数，计算sum(output * w)，相当于加权得标量

In [2]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm()<1000:
    y = y * 2
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
# It actually computes the gradient of sum(y*w)
y.backward(v)
print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


## Part II: Neural Network

In [3]:
import torch.nn as nn
import torch.nn.functional as F

![title](img/Conv2d.png)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
# learnable parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [6]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.1097, -0.0746,  0.0008, -0.0760, -0.0536, -0.0975, -0.0913,  0.0777,
         -0.0453,  0.0321]], grad_fn=<AddmmBackward>)
